# Defensa Técnica: Arquitectura Cognitiva para "VuelaConNosotros"

A continuación, presento la defensa de la arquitectura y la prueba de concepto (PoC) desarrolladas para el agente conversacional "VuelaConNosotros", respondiendo directamente a los tres puntos clave solicitados en el desafío.

## 1. Arquitectura Cognitiva de Alto Nivel

La arquitectura propuesta está diseñada para ser **modular, escalable y observable**, utilizando un orquestador de grafos de estado para gestionar la lógica conversacional de manera explícita y predecible.

### Diagrama de la Arquitectura

```mermaid
graph TD
    subgraph "User Interaction"
        U[👤 User]
    end

    subgraph "API Layer (FastAPI)"
        API[🌐 /vuelaconnosotros/chat]
    end

    subgraph "Cognitive Core (LangGraph)"
        O[⚙️ **StateGraph Orchestrator**]
        
        subgraph "Nodes"
            NLU[🧠 **1. Intent Classifier**<br>Clasifica la intención del usuario]
            Router{<B>2. Intent Router</B><br>Decide el siguiente paso}
            A1[✈️ **FlightStatusAgent**<br>Gestiona consultas de estado]
            A2[🔄 **FlightChangeAgent**<br>Gestiona cambios de vuelo]
            A3[💬 **DefaultAgent**<br>Maneja saludos y respuestas generales]
        end

        subgraph "Tools (Simulated Services)"
            T1[🛠️ get_flight_status]
            T2[🛠️ get_flight_details]
            T3[🛠️ check_flight_availability]
        end
    end

    subgraph "Observability"
        LS[📊 LangSmith Tracing]
    end

    U --> API
    API --> O
    O --> NLU
    NLU --> Router
    Router -- "consultar_estado_vuelo" --> A1
    Router -- "cambiar_vuelo" --> A2
    Router -- "saludo/despedida/otro" --> A3
    
    A1 --> T1
    A2 --> T2 & T3
    
    T1 --> A1
    T2 --> A2
    T3 --> A2

    A1 --> O
    A2 --> O
    A3 --> O

    O --> API
    API --> U

    O --trace--> LS
    NLU --trace--> LS
    A1 --trace--> LS
    A2 --trace--> LS
```

### Componentes Principales y su Interacción

1.  **API Layer (FastAPI):** Es la única puerta de entrada. Recibe la solicitud del usuario, la pasa al orquestador y devuelve la respuesta final. Abstrae toda la lógica cognitiva interna.

2.  **Orquestador (LangGraph):** Es el cerebro del sistema. No es un LLM, sino un **grafo de estados** que define el flujo de la lógica. Su ciclo es:
    *   **Estado Inicial:** Recibe el mensaje del usuario.
    *   **Nodo 1 - `Intent Classifier`:** Un LLM (Gemini Flash) clasifica la intención (`consultar_estado_vuelo`, `cambiar_vuelo`, `saludo`, etc.). Esta es la única inferencia no determinista al inicio del flujo.
    *   **Enrutador Condicional:** Basado en la intención, el grafo dirige el estado al agente correspondiente. Esta es una decisión **determinista** que hace el sistema robusto.
    *   **Nodo 3 - Agentes Especializados:** Se activa el agente responsable. Cada agente es un sub-grafo o una función que puede llamar a herramientas específicas para cumplir su tarea.
    *   **Estado Final:** El agente actualiza el estado con su respuesta y el grafo termina, devolviendo el control a la API.

3.  **Agentes Especializados:** Son los "trabajadores" del sistema. En lugar de un único agente monolítico, he implementado agentes que siguen el Principio de Responsabilidad Única:
    *   `FlightStatusAgent`: Solo puede consultar estados de vuelo.
    *   `FlightChangeAgent`: Orquesta un sub-flujo más complejo para guiar al usuario en la recopilación de datos (vuelo actual, nuevo destino, fecha).
    *   `DefaultAgent`: Maneja interacciones simples como saludos o despedidas sin necesidad de herramientas.

4.  **Herramientas:** Son funciones atómicas que simulan la conexión con sistemas externos (APIs de aerolíneas, bases de datos). Los agentes las utilizan para obtener o modificar información.

5.  **Observabilidad (LangSmith):** Cada paso (clasificación, enrutamiento, llamada a herramienta) se traza automáticamente, permitiendo una depuración y análisis de rendimiento extremadamente detallados.

## 2. Manejo de Intenciones Clave y Desafíos Conversacionales

Las dos intenciones críticas identificadas y para las cuales se implementaron agentes son: **`consultar_estado_vuelo`** y **`cambiar_vuelo`**.

### Flujo Conversacional: `cambiar_vuelo`

Esta intención es más compleja porque requiere múltiples interacciones. Mi `FlightChangeAgent` está diseñado como una **máquina de estados conversacional** para manejar esto.

**Camino Exitoso:**

```
[USER]: "Hola, quiero cambiar mi vuelo VW123 a nombre de Juan Pérez."
 |
[BOT]: (NLU detecta 'cambiar_vuelo' y extrae 'VW123' y 'Juan Pérez'. Llama a la herramienta `get_flight_details` y verifica la reserva)
 |     "¡Hola Juan Pérez! Encontré tu reserva para el vuelo VW123 de BUE a MAD. Para continuar, ¿a qué destino y en qué fecha te gustaría viajar? (ej. a Miami el 2025-09-20)"
 |
[USER]: "Quisiera ir a Miami el 2025-09-20"
 |
[BOT]: (Agente extrae destino y fecha. Llama a la herramienta `check_flight_availability`)
 |     "Perfecto. He encontrado 2 vuelos disponibles para esa fecha: el VW789 a las 22:00 y el VW987 a las 23:30. ¿Te gustaría que proceda con la simulación de cambio para alguna de estas opciones?"
```

**Camino con Aclaración (Manejo de Error):**

```
[USER]: "Quiero cambiar mi vuelo."
 |
[BOT]: (NLU detecta 'cambiar_vuelo', pero no hay datos suficientes. El agente inicia el flujo de recopilación)
 |     "Por supuesto, para cambiar tu vuelo, primero necesito tu número de vuelo."
 |
[USER]: "Es el VW999"
 |
[BOT]: (Agente intenta verificar, pero la herramienta `get_flight_details` no encuentra el vuelo)
 |     "No pude encontrar una reserva con ese número de vuelo. ¿Podrías verificarlo e intentarlo de nuevo?"
```

### Desafío Conversacional: **Gestión de Contexto en Tareas de Múltiples Pasos**

*   **Desafío:** El principal reto en un cambio de vuelo es que el usuario puede proporcionar la información en partes, cambiar de opinión o hacer una pregunta no relacionada en medio del proceso. Un agente sin estado se "perdería" y reiniciaría el flujo, frustrando al usuario.

*   **Mitigación con mi Arquitectura:**
    1.  **Estado Persistente:** El `StateGraph` de LangGraph mantiene el estado de la conversación (`flight_number`, `passenger_name`, `flight_details_verified`, etc.) entre turnos. Si el usuario se desvía, el estado se conserva.
    2.  **Agente como Máquina de Estados:** Mi `FlightChangeAgent` no es solo una función, sino una pequeña máquina de estados. Comprueba qué información ya tiene y qué le falta, haciendo siempre la pregunta correcta para avanzar en el proceso. Esto evita que el bot pregunte por datos que el usuario ya proporcionó.
    3.  **Enrutamiento Claro:** Si el usuario cambia de intención abruptamente (ej. "ok, y cuál es el estado del vuelo AB456?"), el orquestador principal puede pausar el flujo de cambio, enrutar al `FlightStatusAgent`, y luego potencialmente regresar al flujo de cambio, ya que el estado original sigue guardado.

## 3. Estrategia de Evolución y Escalabilidad

La arquitectura está diseñada desde cero para crecer de manera ordenada.

*   **Nuevas Funcionalidades (Escalabilidad Funcional):**
    *   Para añadir una nueva capacidad (ej. "reservar hotel"), el proceso es simple y no afecta a los agentes existentes:
        1.  Crear un nuevo `HotelAgent` en `agents/`.
        2.  Desarrollar sus herramientas (`search_hotels`, `book_room`) en `tools/`.
        3.  Añadir la nueva intención `reservar_hotel` al clasificador.
        4.  Agregar una nueva rama en el enrutador del `chatbot_graph.py`.
    *   Este enfoque de "plug-and-play" evita la complejidad exponencial de los sistemas monolíticos.

*   **Manejo de Más Usuarios (Escalabilidad de Rendimiento):**
    *   La API de FastAPI se puede escalar horizontalmente con cualquier orquestador de contenedores (como Kubernetes) para manejar un mayor volumen de solicitudes.
    *   El estado de la conversación se gestiona con `MemorySaver` de LangGraph, que puede ser reemplazado por un backend más robusto como Redis o una base de datos SQL/NoSQL para manejar sesiones concurrentes a gran escala sin pérdida de datos.

*   **Evolución Continua (Mejora de la Inteligencia):**
    *   **Evaluación y Re-entrenamiento:** Gracias a las trazas de LangSmith, podemos identificar conversaciones fallidas o donde el bot tuvo un rendimiento subóptimo. Estos casos se convierten en datos para un "golden dataset" de evaluación.
    *   **Mejora de Agentes:** Podemos mejorar o reemplazar cualquier componente de forma aislada. Si la clasificación de intención falla, podemos hacer fine-tuning del clasificador sin tocar los agentes. Si un agente es ineficiente, podemos rediseñar su lógica interna sin afectar al resto del sistema.
    *   **Actualización de Herramientas:** Si una API externa cambia, solo necesitamos actualizar la función de la herramienta correspondiente, y todos los agentes que la usan se beneficiarán del cambio sin necesidad de ser modificados.

## 4. Instrucciones Técnicas para la Puesta en Marcha

Para evaluar la solución, he preparado un entorno de prueba completo.

**Requisitos Previos:**
*   Python 3.10+ y `uv` (o `pip`).
*   Clave de API de Google AI y LangSmith.

**Pasos para la Ejecución:**
1.  **Configurar Entorno:**
    *   `cd apps/itti-backend`
    *   Crear y rellenar `.env` a partir de `.env.example`.
2.  **Instalar Dependencias:**
    *   `uv venv` y `uv pip install -r requirements.txt`.
3.  **Iniciar Servidor:**
    *   `uvicorn itti_backend.main:app --host 0.0.0.0 --port 8003 --reload`
4.  **Ejecutar Demostración:**
    *   En otra terminal: `python run_evaluation.py`
    *   Este script simula conversaciones completas y valida la robustez del agente.
5.  **Verificar Trazas:**
    *   Acceder a LangSmith para inspeccionar las trazas detalladas de cada interacción.